# Описание проекта по SQL

Заказчик: стартап, который купил крупный сервис для чтения книг по подписке и планирует развивать этот проект. 

Наша задача - проанализировать базу данных, в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Для этого необходимо осуществить следующие шаги:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Данные:**

`Таблица books`

Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

`Таблица authors`

Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

`Таблица publishers`

Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

`Таблица ratings`

Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

`Таблица reviews`

Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

### Исследуем данные

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выведем первые строки таблиц для ознакомления с данными:

In [4]:
# таблица с книгами
query = '''
SELECT * 
FROM books 
LIMIT 5
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
# таблица с авторами
query = '''
SELECT * 
FROM authors 
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
# таблица с издательствами
query = '''
SELECT * 
FROM publishers 
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
# таблица с пользовательскими оценками
query = '''
SELECT * 
FROM ratings
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
# таблица с пользовательскими обзорами
query = '''
SELECT * 
FROM reviews
LIMIT 5
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Мы ознакомились с данными, теперь можно приступать к решению задач.

### Решение задач

#### Необходимо посчитать, сколько книг вышло после 1 января 2000 года

In [9]:
query = '''
SELECT COUNT(book_id) AS books_cnt
FROM books
WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,books_cnt
0,819


После 1 января 2000 года вышло 819 книг.

#### Необходимо посчитать количество обзоров и среднюю оценку для каждой книги

In [10]:
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT r.review_id) AS review_cnt,
       ROUND(AVG(rate.rating), 2) AS rating_avg
FROM books AS b
LEFT OUTER JOIN ratings AS rate ON b.book_id = rate.book_id
LEFT OUTER JOIN reviews AS r ON rate.book_id = r.book_id
GROUP BY b.book_id, b.title
ORDER BY review_cnt DESC
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_cnt,rating_avg
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Мы объединили 3 таблицы (books, reviews, ratings), сгруппировали данные по названию книги и ее id, затем рассчитали количество обзоров и среднюю оценку для каждой книги.

#### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query = '''
SELECT p.publisher,
       COUNT(b.book_id) AS books_cnt
FROM publishers AS p
INNER JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher
ORDER BY books_cnt DESC
LIMIT 1
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_cnt
0,Penguin Books,42


Итак, издательством, которое выпустило наибольшее число книг толще 50 страниц, является издательство Penguin Books - 42 книги.   

#### Определить автора с самой высокой средней оценкой книг (для  книг с 50 и более оценками)

In [12]:
query = '''
SELECT a.author AS author,
       ROUND(AVG(r.rating),2) AS rating_avg
FROM authors AS a
JOIN books AS b ON a.author_id = b.author_id
JOIN ratings AS r ON b.book_id = r.book_id
WHERE r.book_id IN (SELECT book_id
                  FROM ratings
                  GROUP BY book_id
                  HAVING COUNT(rating) >= 50)
GROUP BY author              
ORDER BY rating_avg DESC
LIMIT 1
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.29


Автором с самой высокой средней оценкой (для книг с 50 и более оценками) стала Дж.Роулинг. Все любят Гарри Поттера)

#### Необходимо посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [13]:
query = '''
SELECT AVG(r.review_count) AS review_avg
FROM (SELECT username,
      COUNT(review_id) AS review_count
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating) > 48) 
                         GROUP BY username) AS r
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_avg
0,24.0


В среднем 24 обзора пишут пользователи, которые поставили больше 48 оценок 

Итак, по результатам анализа базы данных можно сделать следующие выводы:
- после 1 января 2000 года вышло 819 книг;
- издательством, выпустившим наибольшее число книг толще 50 страниц, является Penguin Books (42 книги);
- автором с самой высокой средней оценкой (для книг с 50 и более оценками) является Дж.Роулинг;
- среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24.